In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import tkinter as tk
from tkinter import ttk, messagebox
import pickle

# Load the saved model and scaler
print("Loading model...")
try:
    model = tf.keras.models.load_model(r"C:\Users\jonla\NBA_Playoffs_Series_Predictor\Models\NBA_Playoff_NN.h5")
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    print("Model loaded and compiled")
except Exception as e:
    print(f"Error loading model: {e}")
    exit(1)

print("Loading scaler...")
try:
    with open(r"C:\Users\jonla\NBA_Playoffs_Series_Predictor\Models\scaler.pkl", 'rb') as f:
        scaler = pickle.load(f)
    print("Scaler loaded")
    print("Scaler statistics:")
    for i, feat in enumerate(['T1-T2_W', 'T1-T2_PIE', 'T1-T2_eFG%', 'T1-T2_OREB%', 'T1-T2_DREB%', 
                             'T1-T2_TS%', 'T1vT2_Off-Def', 'T2vT1_Off-Def']):
        print(f"  {feat}: mean={scaler.mean_[i]:.4f}, std={scaler.scale_[i]:.4f}")
except Exception as e:
    print(f"Error loading scaler: {e}")
    exit(1)

# Load and validate 2024-2025 playoff stats
print("Loading playoff stats...")
try:
    playoff_stats_2025 = pd.read_csv(r"C:\Users\jonla\NBA_Playoffs_Series_Predictor\Data\Advanced_Stats_25.csv")
    required_columns = ['Team', 'W', 'PIE', 'eFG%', 'OREB%', 'DREB%', 'TS%', 'OffRtg', 'DefRtg']
    if not all(col in playoff_stats_2025.columns for col in required_columns):
        raise ValueError("nba_advanced_stats_2025.csv missing required columns")
    # Normalize stats if needed
    for col in ['PIE', 'eFG%', 'OREB%', 'DREB%', 'TS%']:
        if playoff_stats_2025[col].max() > 1.0:
            print(f"Normalizing {col} (dividing by 100)")
            playoff_stats_2025[col] /= 100.0
    print("Playoff stats loaded")
    print(playoff_stats_2025[required_columns].head())
except Exception as e:
    print(f"Error loading playoff stats: {e}")
    exit(1)

# Function to prepare input features for prediction
def prepare_input(team1, team2, stats_df):
    print(f"\nPreparing input for {team1} vs {team2}")
    try:
        t1_stats = stats_df[stats_df['Team'] == team1].iloc[0]
        t2_stats = stats_df[stats_df['Team'] == team2].iloc[0]
        
        # Debug: Print raw stats
        print("Team 1 stats:", t1_stats[required_columns[1:]].to_dict())
        print("Team 2 stats:", t2_stats[required_columns[1:]].to_dict())
        
        # Compute feature differences
        features = {
            'T1-T2_W': t1_stats['W'] - t2_stats['W'],
            'T1-T2_PIE': t1_stats['PIE'] - t2_stats['PIE'],
            'T1-T2_eFG%': t1_stats['eFG%'] - t2_stats['eFG%'],
            'T1-T2_OREB%': t1_stats['OREB%'] - t2_stats['OREB%'],
            'T1-T2_DREB%': t1_stats['DREB%'] - t2_stats['DREB%'],
            'T1-T2_TS%': t1_stats['TS%'] - t2_stats['TS%'],
            'T1vT2_Off-Def': t1_stats['OffRtg'] - t2_stats['DefRtg'],
            'T2vT1_Off-Def': t2_stats['OffRtg'] - t1_stats['DefRtg']
        }
        
        # Debug: Print raw feature values
        print("Raw feature values:")
        for k, v in features.items():
            print(f"  {k}: {v:.4f}")
        
        # Convert to numpy array in the correct order
        inputs = ['T1-T2_W', 'T1-T2_PIE', 'T1-T2_eFG%', 'T1-T2_OREB%', 'T1-T2_DREB%', 
                  'T1-T2_TS%', 'T1vT2_Off-Def', 'T2vT1_Off-Def']
        X = np.array([[features[feat] for feat in inputs]])
        
        # Scale the input
        X_scaled = scaler.transform(X)
        
        # Debug: Check for extreme scaled values before clipping
        print("Scaled feature values (before clipping):")
        for i, feat in enumerate(inputs):
            print(f"  {feat}: {X_scaled[0][i]:.4f}")
            if abs(X_scaled[0][i]) > 5.0:
                print(f"  WARNING: {feat} has extreme scaled value ({X_scaled[0][i]:.4f})")
        
        # Clip scaled values to prevent extreme inputs
        X_scaled = np.clip(X_scaled, -5.0, 5.0)
        
        # Debug: Print scaled feature values after clipping
        print("Scaled feature values (after clipping):")
        for i, feat in enumerate(inputs):
            print(f"  {feat}: {X_scaled[0][i]:.4f}")
        
        return X_scaled
    except Exception as e:
        print(f"Error preparing input: {e}")
        raise

# Function to make prediction with symmetry
def predict_winner(team1, team2):
    print("Making prediction...")
    try:
        # Predict T1 vs T2
        X_scaled_t1_t2 = prepare_input(team1, team2, playoff_stats_2025)
        prob_t2_wins = model.predict(X_scaled_t1_t2, verbose=0)[0][0]
        print(f"Prediction probability (T2 wins, {team2} vs {team1}): {prob_t2_wins:.4f}")
        
        # Predict T2 vs T1
        X_scaled_t2_t1 = prepare_input(team2, team1, playoff_stats_2025)
        prob_t1_wins = model.predict(X_scaled_t2_t1, verbose=0)[0][0]
        print(f"Prediction probability (T2 wins, {team1} vs {team2}): {prob_t1_wins:.4f}")
        
        # Average the probabilities to enforce symmetry
        prob_team2_wins = (prob_t2_wins + (1 - prob_t1_wins)) / 2
        print(f"Averaged probability ({team2} wins): {prob_team2_wins:.4f}")
        
        return prob_team2_wins
    except Exception as e:
        messagebox.showerror("Error", f"Error making prediction: {str(e)}")
        return None

# GUI Application
class NBAPredictorApp:
    def __init__(self, root):
        print("Initializing GUI...")
        self.root = root
        self.root.title("NBA Playoff Predictor")
        self.root.geometry("400x300")
        
        self.root.configure(bg="#f0f0f0")
        self.label_font = ("Arial", 12)
        self.button_font = ("Arial", 10, "bold")
        
        teams = sorted(playoff_stats_2025['Team'].tolist())
        
        tk.Label(root, text="Select Team 1:", font=self.label_font, bg="#f0f0f0").pack(pady=10)
        self.team1_var = tk.StringVar()
        self.team1_dropdown = ttk.Combobox(root, textvariable=self.team1_var, values=teams, state="readonly", width=20)
        self.team1_dropdown.pack(pady=5)
        
        tk.Label(root, text="Select Team 2:", font=self.label_font, bg="#f0f0f0").pack(pady=10)
        self.team2_var = tk.StringVar()
        self.team2_dropdown = ttk.Combobox(root, textvariable=self.team2_var, values=teams, state="readonly", width=20)
        self.team2_dropdown.pack(pady=5)
        
        self.predict_button = tk.Button(root, text="Predict", command=self.make_prediction, 
                                       font=self.button_font, bg="#4CAF50", fg="white")
        self.predict_button.pack(pady=20)
        
        self.result_label = tk.Label(root, text="", font=self.label_font, bg="#f0f0f0", wraplength=350)
        self.result_label.pack(pady=10)
        print("GUI initialized")

    def make_prediction(self):
        team1 = self.team1_var.get()
        team2 = self.team2_var.get()
        
        if not team1 or not team2:
            messagebox.showwarning("Input Error", "Please select both teams.")
            return
        
        if team1 == team2:
            messagebox.showwarning("Input Error", "Please select different teams.")
            return
        
        prob_team2_wins = predict_winner(team1, team2)
        if prob_team2_wins is not None:
            if prob_team2_wins >= .5:
                result = f"Predicted Winner: {team2}\n"
                result += f"Probability of {team2} beating {team1}: {prob_team2_wins*100:.2f}%\n"
            else:
                result = f"Predicted Winner: {team1}\n"
                result += f"Probability of {team1} beating {team2}: {(1-prob_team2_wins)*100:.2f}%"
            self.result_label.config(text=result)

# Run the GUI
if __name__ == "__main__":
    print("Starting GUI mainloop...")
    root = tk.Tk()
    app = NBAPredictorApp(root)
    root.mainloop()
    print("GUI closed")

Loading model...
Model loaded and compiled
Loading scaler...
Scaler loaded
Scaler statistics:
  T1-T2_W: mean=0.0000, std=10.4425
  T1-T2_PIE: mean=0.0000, std=3.2917
  T1-T2_eFG%: mean=0.0000, std=2.4626
  T1-T2_OREB%: mean=0.0000, std=3.3072
  T1-T2_DREB%: mean=0.0000, std=2.0915
  T1-T2_TS%: mean=0.0000, std=2.2894
  T1vT2_Off-Def: mean=4.3228, std=3.4889
  T2vT1_Off-Def: mean=4.3228, std=3.4889
Loading playoff stats...
Normalizing PIE (dividing by 100)
Normalizing eFG% (dividing by 100)
Normalizing OREB% (dividing by 100)
Normalizing DREB% (dividing by 100)
Normalizing TS% (dividing by 100)
Playoff stats loaded
                    Team   W    PIE   eFG%  OREB%  DREB%    TS%  OffRtg  \
0    Cleveland Cavaliers  64  0.546  0.578  0.296  0.700  0.607   121.0   
1         Boston Celtics  61  0.542  0.561  0.291  0.717  0.591   119.5   
2  Oklahoma City Thunder  68  0.562  0.560  0.281  0.704  0.593   119.2   
3         Denver Nuggets  50  0.529  0.573  0.311  0.709  0.604   118.9   
4 